Построение дерева решений

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import warnings
warnings.simplefilter('ignore')

import plotly.express as px

In [2]:
df = pd.read_csv('Data/train.csv')
df_test = pd.read_csv('Data/test.csv')

In [3]:
# Реализуем класс узла

class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

In [4]:
# И класс терминального узла (листа)

class Leaf:
    
    def __init__(self, data, values):
        self.data = data
        self.values = values
        self.prediction = self.predict()
        
    def predict(self):
        # подсчет количества объектов разных классов
        prediction = self.values.mean()
        return prediction        

In [5]:
# Расчет качества

def quality(left_values, right_values):

    
    return left_values.var() + right_values.var()

In [6]:
# Разбиение датасета в узле

def split(data, values, index, t):
    
#     print(data.shape, values.shape, index, t)
    
    left = np.where(data[:, index] <= t)
    right = np.where(data[:, index] > t)
        
    true_data = data[left]
    false_data = data[right]
    true_values = values[left]
    false_values = values[right]
        
    return true_data, false_data, true_values, false_values

In [7]:
# Нахождение наилучшего разбиения

def find_best_split(data, values):
    
    #  обозначим минимальное количество объектов в узле
    min_leaf = 5

    best_quality = values.var() * 1e+5
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    
    for index in range(n_features):
        t_values = [row[index] for row in data]
#         print(t_values)
        for t in t_values:
#             print(t)
            true_data, false_data, true_values, false_values = split(data, values, index, t)
            #  пропускаем разбиения, в которых в узле остается менее 5 объектов
            if len(true_data) < min_leaf or len(false_data) < min_leaf:
                continue
            
            current_quality = quality(true_values, false_values)
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_quality < best_quality:
                best_quality, best_t, best_index = current_quality, t, index

    return best_quality, best_t, best_index

In [8]:
# Построение дерева с помощью рекурсивной функции

def build_tree(data, values, max_depth):

    quality, t, index = find_best_split(data, values)

    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
    if quality == 0:
        return Leaf(data, values)
    elif values.var() == 0:
        return Leaf(data, values)
    elif data.shape[0] < 15:
        return Leaf(data, values)
    elif max_depth == 0:
        return Leaf(data, values)
        

    true_data, false_data, true_values, false_values = split(data, values, index, t)
    max_depth -= 1
    # Рекурсивно строим два поддерева
    true_branch = build_tree(true_data, true_values, max_depth)
    false_branch = build_tree(false_data, false_values, max_depth)

    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

In [9]:
def classify_object(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)

In [10]:
def predict(data, tree):
    
    values = []
    for obj in data:
        prediction = classify_object(obj, tree)
        values.append(prediction)
    return values

In [11]:
X = df[df.columns[1:-1]].to_numpy()
y = df[df.columns[-1]].to_numpy()

In [12]:
# Разобьем выборку на обучающую и тестовую

from sklearn import model_selection

train_data, test_data, train_values, test_values = model_selection.train_test_split(X, y, test_size = 0.3, random_state = 1)

In [13]:
%%time
# Построим дерево по обучающей выборке
my_tree = build_tree(train_data, train_values, 3)

Wall time: 38.7 s


In [14]:
# Получим ответы для обучающей выборки 
train_answers = predict(train_data, my_tree)

In [15]:
# И получим ответы для тестовой выборки
answers = predict(test_data, my_tree)

In [16]:
# Введем функцию подсчета точности R2
def r2(y_real, y_pred):
    r2 = 1 - np.sum(np.square(y_real - y_pred))/np.sum(np.square(y_real - y_real.mean()))
    return r2

In [17]:
# Точность на обучающей выборке
train_accuracy = r2(train_values, train_answers)
train_accuracy

0.6173435960072137

In [18]:
# Точность на тестовой выборке
test_accuracy = r2(test_values, answers)
test_accuracy

0.6007861525667798

In [19]:
def gb_predict(X, trees_list, eta):
    answers = np.zeros(X.shape[0])
    for tree in trees_list:
        answers_new = predict(X, tree)
        answers = answers + np.array(answers_new)*eta
    
    return answers

Напишем производную от функции ошибки MSE, для того чтобы посчитать антиградиент

In [20]:
def bias(y, z):
    return (y - z)

In [21]:
def first_val(y_true):
    y_true[:] = y_true.mean()
    return 

In [22]:
def gb_fit(n_trees, train_data, test_data, train_values, test_values, eta, max_depth):
    
    # Деревья будем записывать в список
    trees = []
    
    # Будем записывать ошибки на обучающей и тестовой выборке на каждой итерации в список
    train_errors = []
    test_errors = []
    
    for i in range(n_trees):
        print('Итерация ', i, end='\r')
#         tree = build_tree()

        # инициализируем бустинг начальным алгоритмом, возвращающим ноль, 
        # поэтому первый алгоритм просто обучаем на выборке и добавляем в список
        if len(trees) == 0:
            # обучаем первое дерево на обучающей выборке
            tree = build_tree(train_data, train_values, max_depth)
            print('Ошибка на итерации {} = {}'.format(i, r2(train_values, gb_predict(train_data, trees, eta))))
            
            train_errors.append(r2(train_values, gb_predict(train_data, trees, eta)))
            test_errors.append(r2(test_values, gb_predict(test_data, trees, eta)))
#             print('Дерево ', i)
#             evaluate_alg_1(X_train, X_test, y_train, y_test, tree, eta)
        else:
            # Получим ответы на текущей композиции
            target = gb_predict(train_data, trees, eta)
            
            # алгоритмы начиная со второго обучаем на сдвиг
            tree = build_tree(train_data, bias(train_values, target), max_depth)
#             print(bias(y_train, target))

            print('Ошибка на итерации {} = {}'.format(i, r2(train_values, gb_predict(train_data, trees, eta))))
            print('Ошибка на тесте на итерации {} = {}'.format(i, r2(test_values, gb_predict(test_data, trees, eta))))
            train_errors.append(r2(train_values, gb_predict(train_data, trees, eta)))
            test_errors.append(r2(test_values, gb_predict(test_data, trees, eta)))

        trees.append(tree)
        
#         evaluate_alg_1(X_train, X_test, y_train, y_test, trees, eta)
    last_tree = trees[-1]
        
        
    return trees, train_errors, test_errors, last_tree

In [23]:
def evaluate_alg(X_train, X_test, y_train, y_test, trees, eta):
    train_prediction = gb_predict(X_train, trees, eta)

    print(f'Ошибка алгоритма из {n_trees} деревьев  \
    с шагом {eta} на тренировочной выборке: {r2(y_train, train_prediction)}')

    test_prediction = gb_predict(X_test, trees, eta)

    print(f'Ошибка алгоритма из {n_trees} деревьев  \
    с шагом {eta} на тестовой выборке: {r2(y_test, test_prediction)}')

In [ ]:
%%time
# Число деревьев в ансамбле
n_trees = 50

# глубина деревьев
max_depth = 3

# Шаг
eta = 0.5

trees, train_errors, test_errors, last_tree = gb_fit(n_trees, train_data, test_data, train_values, test_values, eta, max_depth)

Ошибка на итерации 0 = -22.466397322351927
Ошибка на итерации 1 = -5.153591633582571
Ошибка на тесте на итерации 1 = -5.244008160875516
Ошибка на итерации 2 = -0.9384799334392897
Ошибка на тесте на итерации 2 = -0.9407184578622674
Ошибка на итерации 3 = 0.11529799159653087
Ошибка на тесте на итерации 3 = 0.12222550868684479
Ошибка на итерации 4 = 0.37874247285548546
Ошибка на тесте на итерации 4 = 0.38152227072202294
Ошибка на итерации 5 = 0.4446035931702246
Ошибка на тесте на итерации 5 = 0.44312684642976763
Ошибка на итерации 6 = 0.4610688732489092
Ошибка на тесте на итерации 6 = 0.45691818295617914
Ошибка на итерации 7 = 0.4651851932685803
Ошибка на тесте на итерации 7 = 0.45956111338751915
Ошибка на итерации 8 = 0.4662142732734982
Ошибка на тесте на итерации 8 = 0.459819394145223
Ошибка на итерации 9 = 0.4664715432747276
Ошибка на тесте на итерации 9 = 0.4596827384095834
Ошибка на итерации 10 = 0.46653586077503517
Ошибка на тесте на итерации 10 = 0.4595479615131408
Ошибка на итерац

In [25]:
evaluate_alg(train_data, test_data, train_values, test_values, trees, eta)

Ошибка алгоритма из 20 деревьев      с шагом 0.5 на тренировочной выборке: 0.4665572999213582
Ошибка алгоритма из 20 деревьев      с шагом 0.5 на тестовой выборке: 0.45936908175285185
